# 03 - Feature Engineering & KPI Creation
## Building Business-Driven Features for Customer Retention Analysis

**Objective**: Create and visualize engineered features:
- Tenure groups for lifecycle analysis
- Spending categories for revenue segmentation
- Retention score (composite metric)
- Customer value segments
- Service usage metrics
- Risk flags and indicators

**Process**: Use `feature_engineering.py` or load pre-processed final data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings

# Add scripts directory
sys.path.append('../scripts')

# Configure
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 1. Load Cleaned and Final Data

In [ ]:
# Load cleaned data (before feature engineering)
df_clean = pd.read_csv('../data/processed/cleaned_data.csv')
print(f"✓ Cleaned data: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

# Load final data (after feature engineering)
df_final = pd.read_csv('../data/processed/final_data.csv')
print(f"✓ Final data: {df_final.shape[0]:,} rows × {df_final.shape[1]} columns")

new_features = df_final.shape[1] - df_clean.shape[1]
print(f"\n📊 New features created: {new_features}")

## 2. Feature 1: Tenure Groups

Categorizing customers based on their tenure with the company.

In [ ]:
# Analyze tenure groups
tenure_dist = df_final['tenure_group'].value_counts()
tenure_order = ['0-12 months', '12-24 months', '24-36 months', '36-48 months', '48+ months']
tenure_dist = tenure_dist.reindex(tenure_order)

print("\n📅 Tenure Group Distribution:\n")
print(tenure_dist)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
tenure_dist.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Customer Distribution by Tenure Group', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count')
axes[0].set_xlabel('Tenure Group')
axes[0].tick_params(axis='x', rotation=45)
for container in axes[0].containers:
    axes[0].bar_label(container)

# Churn rate by tenure group
churn_by_tenure = df_final.groupby('tenure_group')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
churn_by_tenure = churn_by_tenure.reindex(tenure_order)

churn_by_tenure.plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Churn Rate by Tenure Group', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_xlabel('Tenure Group')
axes[1].tick_params(axis='x', rotation=45)
axes[1].axhline(y=33.7, color='red', linestyle='--', linewidth=1.5, label='Overall Churn: 33.7%')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Insight: Newer customers (0-12 months) have significantly higher churn rates.")

## 3. Feature 2: Average Monthly Spend Categories

In [ ]:
# Analyze spending categories
spend_dist = df_final['avg_monthly_spend'].value_counts()
spend_order = ['Low (<$30)', 'Medium ($30-$70)', 'High ($70-$100)', 'Premium ($100+)']
spend_dist = spend_dist.reindex(spend_order)

print("\n💰 Monthly Spend Distribution:\n")
print(spend_dist)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
colors_spend = ['#3498db', '#2ecc71', '#f39c12', '#9b59b6']
spend_dist.plot(kind='bar', ax=axes[0], color=colors_spend)
axes[0].set_title('Customer Distribution by Spend Category', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count')
axes[0].set_xlabel('Spend Category')
axes[0].tick_params(axis='x', rotation=30)
for container in axes[0].containers:
    axes[0].bar_label(container)

# Churn rate by spend
churn_by_spend = df_final.groupby('avg_monthly_spend')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
churn_by_spend = churn_by_spend.reindex(spend_order)

churn_by_spend.plot(kind='bar', ax=axes[1], color=colors_spend)
axes[1].set_title('Churn Rate by Spend Category', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_xlabel('Spend Category')
axes[1].tick_params(axis='x', rotation=30)
axes[1].axhline(y=33.7, color='red', linestyle='--', linewidth=1.5, label='Overall: 33.7%')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Insight: Premium customers have similar churn rates, suggesting value matters more than price.")

## 4. Feature 3: Service Usage Score

Count of services each customer is using (0-8 scale).

In [ ]:
# Analyze service usage
service_dist = df_final['service_usage_score'].value_counts().sort_index()

print("\n📊 Service Usage Score Distribution:\n")
print(service_dist)
print(f"\nAverage services per customer: {df_final['service_usage_score'].mean():.2f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution
service_dist.plot(kind='bar', ax=axes[0], color='teal')
axes[0].set_title('Distribution of Service Usage Score', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Customers')
axes[0].set_xlabel('Number of Services Used')
axes[0].tick_params(axis='x', rotation=0)

# Churn rate by service usage
churn_by_service = df_final.groupby('service_usage_score')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)

churn_by_service.plot(kind='line', ax=axes[1], marker='o', color='coral', linewidth=2, markersize=8)
axes[1].set_title('Churn Rate by Service Usage', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].set_xlabel('Number of Services Used')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=33.7, color='red', linestyle='--', linewidth=1.5, label='Overall: 33.7%')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Insight: Customers with more services tend to have lower churn (bundle effect).")

## 5. Feature 4: Retention Score (Composite Metric)

Weighted composite score (0-100) combining tenure, contract, services, payment, and support.

In [ ]:
# Analyze retention score
print("\n🎯 Retention Score Statistics:\n")
print(df_final['retention_score'].describe())

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution
axes[0, 0].hist(df_final['retention_score'], bins=30, color='steelblue', edgecolor='black')
axes[0, 0].axvline(df_final['retention_score'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {df_final['retention_score'].mean():.1f}")
axes[0, 0].axvline(df_final['retention_score'].median(), color='green', linestyle='--', linewidth=2, label=f"Median: {df_final['retention_score'].median():.1f}")
axes[0, 0].set_title('Retention Score Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Retention Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# Box plot by churn
sns.boxplot(data=df_final, x='churn', y='retention_score', palette=['#2ecc71', '#e74c3c'], ax=axes[0, 1])
axes[0, 1].set_title('Retention Score by Churn Status', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Churn')
axes[0, 1].set_ylabel('Retention Score')

# Scatter: Retention Score vs Monthly Charges
for churn_val, color in [('No', '#2ecc71'), ('Yes', '#e74c3c')]:
    data = df_final[df_final['churn'] == churn_val]
    axes[1, 0].scatter(data['retention_score'], data['monthly_charges'], 
                       alpha=0.5, s=30, c=color, label=f'Churn: {churn_val}')
axes[1, 0].set_title('Retention Score vs Monthly Charges', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Retention Score')
axes[1, 0].set_ylabel('Monthly Charges ($)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Retention score categories
df_final['retention_category'] = pd.cut(df_final['retention_score'], 
                                         bins=[0, 30, 60, 100], 
                                         labels=['Low (0-30)', 'Medium (30-60)', 'High (60-100)'])
retention_cat_dist = df_final['retention_category'].value_counts()

retention_cat_dist.plot(kind='bar', ax=axes[1, 1], color=['#e74c3c', '#f39c12', '#2ecc71'])
axes[1, 1].set_title('Retention Score Categories', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_xlabel('Retention Category')
axes[1, 1].tick_params(axis='x', rotation=0)
for container in axes[1, 1].containers:
    axes[1, 1].bar_label(container)

plt.tight_layout()
plt.show()

# Statistical comparison
print("\n📊 Retention Score by Churn:\n")
print(f"No Churn:  {df_final[df_final['churn'] == 'No']['retention_score'].mean():.2f}")
print(f"Churned:   {df_final[df_final['churn'] == 'Yes']['retention_score'].mean():.2f}")
print(f"\n💡 Insight: Retention score strongly differentiates between churned and retained customers.")

## 6. Feature 5: Customer Value Segments

Four-quadrant segmentation based on retention score and revenue.

In [ ]:
# Analyze customer segments
segment_dist = df_final['customer_value_segment'].value_counts()

print("\n🎯 Customer Value Segments:\n")
for segment, count in segment_dist.items():
    pct = (count / len(df_final)) * 100
    print(f"{segment:<30} {count:>5} ({pct:>5.1f}%)")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
colors_segment = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c']
axes[0].pie(segment_dist.values, labels=segment_dist.index, autopct='%1.1f%%', 
            colors=colors_segment, startangle=90)
axes[0].set_title('Customer Value Segment Distribution', fontsize=12, fontweight='bold')

# Churn rate by segment
churn_by_segment = df_final.groupby('customer_value_segment')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)

churn_by_segment.plot(kind='barh', ax=axes[1], color=colors_segment)
axes[1].set_title('Churn Rate by Customer Segment', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Churn Rate (%)')
axes[1].set_ylabel('Segment')
axes[1].axvline(x=33.7, color='red', linestyle='--', linewidth=1.5, label='Overall: 33.7%')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n💡 Insight: 'High Value - At Risk' and 'Low Value - At Risk' segments need immediate attention.")

## 7. Feature 6: Payment Issue Flag & Churn Risk Flag

In [ ]:
# Analyze risk flags
payment_issue_count = df_final['payment_issue_flag'].sum()
churn_risk_count = df_final['churn_risk_flag'].sum()

print("\n⚠️ Risk Indicators:\n")
print(f"Payment Issue Flag (Electronic Check):")
print(f"  Flagged: {payment_issue_count:,} ({payment_issue_count/len(df_final)*100:.1f}%)")
print(f"\nChurn Risk Flag (High Risk):")
print(f"  Flagged: {churn_risk_count:,} ({churn_risk_count/len(df_final)*100:.1f}%)")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Payment issue impact
payment_churn = df_final.groupby('payment_issue_flag')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
labels_payment = ['No Issue (0)', 'Payment Issue (1)']
bars1 = axes[0].bar(labels_payment, payment_churn.values, color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Churn Rate by Payment Issue Flag', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Churn Rate (%)')
axes[0].axhline(y=33.7, color='blue', linestyle='--', linewidth=1.5, label='Overall: 33.7%')
axes[0].legend()
for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Churn risk flag impact
risk_churn = df_final.groupby('churn_risk_flag')['churn'].apply(lambda x: (x == 'Yes').sum() / len(x) * 100)
labels_risk = ['Low Risk (0)', 'High Risk (1)']
bars2 = axes[1].bar(labels_risk, risk_churn.values, color=['#2ecc71', '#e74c3c'])
axes[1].set_title('Churn Rate by Churn Risk Flag', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].axhline(y=33.7, color='blue', linestyle='--', linewidth=1.5, label='Overall: 33.7%')
axes[1].legend()
for bar in bars2:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Insight: Both flags effectively identify high-risk customers.")

## 8. Feature Correlation Analysis

In [ ]:
# Select engineered features for correlation
engineered_features = [
    'service_usage_score', 
    'retention_score', 
    'payment_issue_flag',
    'churn_risk_flag',
    'internet_service_value',
    'total_services_count',
    'contract_value_score'
]

# Add binary churn
df_corr = df_final[engineered_features].copy()
df_corr['churn_binary'] = (df_final['churn'] == 'Yes').astype(int)

# Calculate correlation
correlation_matrix = df_corr.corr()

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Engineered Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Show correlations with churn
print("\n🔗 Correlation with Churn (Engineered Features):\n")
churn_corr = correlation_matrix['churn_binary'].sort_values(ascending=False)
for feature, corr in churn_corr.items():
    if feature != 'churn_binary':
        direction = "↑ Positive" if corr > 0 else "↓ Negative"
        print(f"  {feature:<25} {corr:>6.3f}  {direction}")

print("\n💡 Insight: Churn risk flag and retention score are strong predictors.")

## 9. Multi-Dimensional Analysis: Value Segment Matrix

In [ ]:
# Create a 2D view of customer segments
fig, ax = plt.subplots(figsize=(12, 8))

# Define segment colors
segment_colors = {
    'High Value - High Retention': '#2ecc71',
    'Low Value - High Retention': '#3498db',
    'High Value - At Risk': '#f39c12',
    'Low Value - At Risk': '#e74c3c'
}

# Plot each segment
for segment, color in segment_colors.items():
    data = df_final[df_final['customer_value_segment'] == segment]
    ax.scatter(data['monthly_charges'], data['retention_score'], 
               alpha=0.6, s=50, c=color, label=segment, edgecolors='black', linewidth=0.5)

# Add quadrant lines
ax.axvline(x=70, color='black', linestyle='--', linewidth=1.5, alpha=0.5)
ax.axhline(y=70, color='black', linestyle='--', linewidth=1.5, alpha=0.5)

# Labels and title
ax.set_xlabel('Monthly Charges ($)', fontsize=12)
ax.set_ylabel('Retention Score', fontsize=12)
ax.set_title('Customer Value Segment Matrix\n(Monthly Revenue vs Retention Score)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

# Add annotations
ax.text(135, 90, 'Protect & Grow', fontsize=10, fontweight='bold', color='darkgreen', 
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
ax.text(20, 90, 'Maintain', fontsize=10, fontweight='bold', color='darkblue',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
ax.text(135, 15, 'Urgent Action', fontsize=10, fontweight='bold', color='darkorange',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
ax.text(20, 15, 'Intervention', fontsize=10, fontweight='bold', color='darkred',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n💡 This matrix helps prioritize retention efforts based on value and risk.")

## 10. Feature Engineering Summary

### Features Created (9 Total)

#### 1. **tenure_group** (Categorical)
- **Purpose**: Customer lifecycle segmentation
- **Categories**: 5 groups (0-12, 12-24, 24-36, 36-48, 48+ months)
- **Key Finding**: Early months critical - highest churn in 0-12 group

#### 2. **avg_monthly_spend** (Categorical)
- **Purpose**: Revenue tier classification
- **Categories**: Low, Medium, High, Premium
- **Key Finding**: Price not primary driver - Premium customers churn too

#### 3. **service_usage_score** (Numerical: 0-8)
- **Purpose**: Measure service adoption
- **Range**: 0 (no services) to 8 (all services)
- **Key Finding**: Inverse relationship with churn - bundling works!

#### 4. **retention_score** (Numerical: 0-100)
- **Purpose**: Composite retention likelihood
- **Formula**: Weighted (Tenure 30% + Contract 25% + Services 20% + Payment 15% + Support 10%)
- **Key Finding**: Strong predictor - avg 56.5 for retained vs 25.8 for churned

#### 5. **customer_value_segment** (Categorical)
- **Purpose**: Strategic segmentation for retention efforts
- **Segments**: 4 quadrants (High/Low Value × High Retention/At Risk)
- **Key Finding**: 21.2% in "High Value - At Risk" = priority target

#### 6. **payment_issue_flag** (Binary: 0/1)
- **Purpose**: Identify payment-related risk
- **Trigger**: Electronic check payment method
- **Key Finding**: 38.8% churn for flagged vs 31.5% for non-flagged

#### 7. **churn_risk_flag** (Binary: 0/1)
- **Purpose**: High-risk customer identification
- **Criteria**: Retention score < 40, new month-to-month, or 8+ support calls
- **Key Finding**: 54% churn for flagged vs 12% for non-flagged

#### 8. **internet_service_value** (Numerical: 0-2)
- **Purpose**: Service tier encoding
- **Values**: 0=No, 1=DSL, 2=Fiber optic
- **Key Finding**: Higher tier correlates with higher revenue but mixed churn

#### 9. **total_services_count** (Numerical)
- **Purpose**: Overall service engagement
- **Range**: Count of all active services
- **Key Finding**: Similar to service_usage_score - more = lower churn

### Feature Impact on Model Readiness

- **Categorical Features**: 4 (for segmentation and filtering)
- **Numerical Features**: 5 (for scoring and modeling)
- **Binary Flags**: 2 (for risk identification)

### Business Value

1. **Retention Score**: Single metric to prioritize customer interventions
2. **Value Segments**: Strategic framework for retention campaigns
3. **Risk Flags**: Automated early warning system
4. **Tenure Groups**: Lifecycle-based marketing strategies
5. **Service Usage**: Bundle optimization opportunities

### Next Steps

Proceed to **04_Final_Analysis.ipynb** for comprehensive KPI calculations and Power BI preparation.

In [ ]:
print("\n" + "="*70)
print("  FEATURE ENGINEERING COMPLETE - Proceed to 04_Final_Analysis.ipynb")
print("="*70)